<h1><center>Ouster Data Loading and Processing Tutorial</center></h1> 

### **Ouster Confidential**

**Author:** `Reyhaneh Kazerani`  
**Email:** `reyhaneh@ouster.io`   
**Date:** Aug 2020   
**Version:** 1.0.0  

This is an interactive toturial to demonstrate how to load and process Ouster Data for a sample Pytorch model consumption.    

First, let's include all the needed functions

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import torch


from PIL import Image, ImageOps, ImageEnhance, PILLOW_VERSION
from torch.utils.data import * 
from torchvision import transforms as T
from torchvision.transforms import functional as F
%matplotlib inline

Some auxiliary functions for reading a data point

In [ ]:
def read_ouster_frames(path, **kwargs):
    """
    Reads and splits the PNG by their channels, blue (ambient),
    green and red (range) and alpha(intensity). Then computes the
    range by combining red and green channel.
    Args:
    path (string) : path to the OSF
    Returns:
    range (array) : 16 bit integers representing range of each pixel
    ambient (array) : memory map of ambient data
    intensity (array) : intensity values
    """
    if kwargs.get('cv2', False):
        intensity = cv2.imread(path['intensity'], cv2.IMREAD_UNCHANGED)
        ambient = cv2.imread(path['ambient'], cv2.IMREAD_UNCHANGED)
        range_ = cv2.imread(path['range'], cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(path['mask'], cv2.IMREAD_GRAYSCALE)
        

        # Channels orders : blue, green, red and alpha
        _, intensity_2nd, intensity_1st = cv2.split(intensity)
        _, ambient_2nd, ambient_1st = cv2.split(ambient)
        range_3rd, range_2nd, range_1st = cv2.split(range_)
        
        

    else:
        intensity = Image.open(path['intensity'])
        ambient = Image.open(path['ambient'])
        range_ = Image.open(path['range'])
        mask = Image.open(path['mask']).convert('P')

        range_1st, range_2nd, range_3rd = range_.split()

    # Calculate range from r and g channels
    # Merges the channels
    range_ = (np.uint32(range_1st) + (np.uint32(range_2nd) << 8) +
           (np.uint32(range_3rd) << 16)) * 1.0

    return range_, intensity, ambient, mask


def normalize(image , percentile=0.03):
    """
    Scales the image so that contrast is stretched between 0 and 1,
    so that the top percentile is 1 and the bottom percentile is 0.
    """
    # Finds the cut off points
    flat_arr = image.reshape(-1)
    indices = flat_arr.nonzero()[0]
    indices_size = indices.shape[0]
    kth_extreme = int(percentile*indices_size)
    lo = np.partition(flat_arr[indices], kth_extreme)[kth_extreme]
    hi = np.partition(flat_arr[indices], indices_size - kth_extreme - 1)[indices_size - kth_extreme - 1]

    # Normalizes the image based on the low and high cut offs
    image = image.astype(np.float) - lo
    image *= (1.0/(hi - lo))
    image = image.clip(min=0,max=1)
    return image

Some transformations for transforming a data point to tensor, compose different transformations and concatinating different features. These transformations are callable classes instead of simple functions so that parameters of the transform need not be passed everytime it’s called.

In [ ]:
class ToTensor():
    """
    Converts an image (PIL) to a tensor
    Converts and image in (H x W x C) in the range [0, 255] to a
    torch.FloatTensor of shape (C x H x W) in the same range.
    The conversion is inplace.
    """

    def __init__(self, inplace=True):
        self.inplace = inplace

    def __call__(self, data_point, target):
        # Checks for inplace or copy
        if self.inplace:
            new_data_point = data_point
            new_target = target
        else:
            new_data_point = data_point.copy()
            new_target = target.copy()
        # transforms every channel
        for key, value in data_point.items():
            new_data_point[key] = to_tensor(value)
        # transforms the mask
        new_target = torch.as_tensor(np.asarray(target), dtype=torch.int64)
    
        return new_data_point, new_target
    
    
class FeatureConcat():
    """
    Concatenates Ouster specific features.
    If depth feature is used then it casts the tensors to torch.DoubleTensor
    features should already be Tensors
    Args:
     transforms (list of ``Transform`` objects): list of transforms to compose.
    Example:
     >>> data_point = {}
     >>> data_point['intensity'] = torch.randint(low=0, high=255, (1, 512, 64))
     >>> data_point['depth'] = torch.randint(low=0, high=30000, (1, 512, 64), dtype= torch.FloatTensor)
     >>> target = torch.randint(low=0, high=10, (1, 512, 64), dtype=torch.LongTensor)
     >>> transform = transforms.FeatureConcat(['intensity', 'depth'])
     >>> result = transform(data_point, target)
    """

    def __init__(self, feature_names=None):
        if not feature_names:
            self.feature_names = data_point.keys()
        self.feature_names = feature_names

    def __call__(self, data_point, target):
        # Checks for Tensor type
        # FIXME : THIS IS FOR ROS
        #assert torch.is_tensor(
        #    data_point[self.feature_names[0]]), f"Pytorch Tensor was expected"
        if len(self.feature_names) == 1:
            return data_point[self.feature_names[0]], target
        new_data_point = torch.cat(
            [data_point[feature].float() for feature in self.feature_names], dim=0)
        return new_data_point, target
    

class Compose():
    """
    Composes several transforms together.
    Args:
      transforms (list of ``Transform`` objects): list of transforms to compose.
    Example:
      >>> transforms.Compose([
      >>>     transforms.ToTensor(),
      >>>     transforms.FeatureConcat(['features'])
      >>> ])
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, data_point, target):
        for transform in self.transforms:
            data_point, target = transform(data_point, target)
        return data_point, target

    def __repr__(self):
        format_string = self.__class__.__name__ + '('
        for transform in self.transforms:
            format_string += '\n'
            format_string += '    {0}'.format(transform)
        format_string += '\n)'
        return format_string
    
    
def to_tensor(pic):
    """
    Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.
    See ``ToTensor`` for more details.
    Args:
     pic (PIL Image or numpy.ndarray): osf feature (like intensity, depth or ambient) to be converted to tensor.
    Returns:
     Tensor: Converted osf feature.
    """
    if isinstance(pic, np.ndarray):
        # handle numpy array
        if pic.ndim == 2:
            pic = pic[:, :, None]

        img = torch.from_numpy(pic.transpose((2, 0, 1)))
        # backward compatibility
        if isinstance(img, torch.ByteTensor):
            return img.float()  #.div(255)
        else:
            return img

    # handle PIL Image
    if pic.mode == 'I':
        img = torch.from_numpy(np.array(pic, np.int32, copy=False))
    elif pic.mode == 'I;16':
        img = torch.from_numpy(np.array(pic, np.int16, copy=False))
    elif pic.mode == 'F':
        img = torch.from_numpy(np.array(pic, np.float32, copy=False))
    elif pic.mode == '1':
        img = 255 * torch.from_numpy(np.array(pic, np.uint8, copy=False))
    else:
        img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
    # PIL image mode: L, LA, P, I, F, RGB, YCbCr, RGBA, CMYK
    if pic.mode == 'YCbCr':
        nchannel = 3
    elif pic.mode == 'I;16':
        nchannel = 1
    else:
        nchannel = len(pic.mode)
    img = img.view(pic.size[1], pic.size[0], nchannel)
    # put it from HWC to CHW format
    # yikes, this transpose takes 80% of the loading time/CPU
    img = img.transpose(0, 1).transpose(0, 2).contiguous()
    if isinstance(img, torch.ByteTensor):
        return img.float()  #.div(255)
    else:
        return img    

In [ ]:
class OusterDataset(Dataset):
    def __init__(self, data_dir, features, transforms=None):
        # Sets the data directory
        self.data_dir= data_dir
        self.ambient_ls = list(
          sorted(os.listdir(os.path.join(data_dir, "ambient"))))
        self.intensity_ls = list(
          sorted(os.listdir(os.path.join(data_dir, "intensity"))))
        self.range_ls = list(
          sorted(os.listdir(os.path.join(data_dir, "range"))))
        self.masks_ls = list(
          sorted(os.listdir(os.path.join(data_dir, "semantic_masks"))))

        #  Sets the transforms
        self.transforms = self.get_transforms(
          transforms, features=features)  # Preprocessing for data

 
    def __getitem__(self, idx):
        # load images ad masks
        intensity = os.path.join(os.path.join(self.data_dir, "intensity"),
                               self.intensity_ls[idx])
        range_ = os.path.join(os.path.join(self.data_dir, "range"), self.range_ls[idx])
        ambient = os.path.join(os.path.join(self.data_dir, "ambient"), self.ambient_ls[idx])
        mask = os.path.join(self.data_dir, "semantic_masks", self.masks_ls[idx])
        frame_path = {
            'intensity': intensity,
            'ambient': ambient,
            'range': range_,
            'mask': mask
        }

        
        # Reading the osf and its mask
        range_, intensity, ambient, mask = read_ouster_frames(frame_path)
       

        data_point = {}
        data_point["intensity"] = intensity
        data_point["range"] = range_
        data_point["ambient"] = ambient

        if self.transforms is not None:
            data_point, mask = self.transforms(data_point, mask)
        return {'data': data_point, 'mask':mask}
 
    def __len__(self):
        return len(self.intensity_ls)
 
    @staticmethod
    def get_transforms(train=None, **kwargs):
        """
        Composes the transforms. If dataset is not in train mode then it dwould
        only do the primary transfomation for network usage.
        Args:
          train (boolean) : in train mode or not
          transforms (optional, Callable): list of transfomations
        Returns:

        """
        transforms = [] if not train else kwargs('transforms', [])
        transforms.append(ToTensor())
        transforms.append(FeatureConcat(kwargs.get('features', None)))
        return Compose(transforms)

Now let's visualize our data and see if it has been loaded currectly. for that you can easily enter the path to your data folder below. Choose an index in range of your data length, and run the following commands. You can easily choose which features to be added by changing the feature parameter `feature` in `OusterDataset`. You can choose any subset of `intensity`, `ambient` and `range`

In [ ]:
data = './data' # Path to data folder
idx = 0 # Index for a data point
features = ['intensity', 'range', 'ambient'] # A list of features to be processed

# Instantiates an instance of OusterDataset for reading and processing the data
test_data = OusterDataset(data, features=features)


For visualizing the data you need to reformat the tensors to `numpy` arrays. You can follow the following instructions:

In [ ]:
# Converts the tensors to numpy objects
intensity=test_data[idx]['data'][0,:,:].cpu().numpy()
range_=test_data[idx]['data'][1,:,:].cpu().numpy() 
ambient=test_data[idx]['data'][2,:,:].cpu().numpy()
mask=test_data[idx]['mask'].cpu().numpy()

# Visualizes the results
f_image, ax_image = plt.subplots(4, figsize=(15, 5))
ax_image[0].imshow(normalize(intensity))
ax_image[0].set_title('Intensity image')
ax_image[1].imshow(normalize(range_))
ax_image[1].set_title('Range image' )
ax_image[2].imshow(normalize(ambient))
ax_image[2].set_title('Ambient image')
ax_image[3].imshow(normalize(mask))
ax_image[3].set_title('Semantic mask')
plt.show()
